In [1]:
import numpy as np
import pickle
from tqdm import tqdm

### Ok, now that we have our libraries, let's import our data
#### For our first example, we will use a classic dataset "MNIST":
MNIST is one of the most famous dataset that was used as a benchmark for many machine learning models. Currently it is outdated because it was too easy. By the end of the course, the MNIST dataset will appear too easy as well, but for now it is a good start.
#### So what is the MNIST dataset?

This dataset is made of 28 x 28 grayscale images (each pixel is somewhere between black and white, no RGB). Each image represents a hand written english digit. The dataset contains 60000 training examples and 10000 testing examples.
<br>
<br>
![MNIST Dataset](mnist.png)

In [4]:
# Load the dataset from a previously saved file
with open("mnist.pkl",'rb') as f:
        mnist = pickle.load(f)
        
# Divide the dataset into seperate variable for training and testing
# Also the images will be called X and the labels will be called y (example: training_X[0] is the digit 5 and the label "5" is training_y[0])
training_X, training_y, testing_X, testing_y = mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

### Now that we have our data, let's see our first algorithm

## KNN (K-nearest neighbors)
One of the most famous algorithms that is easy to implement and performs very well. To start, we will discuss the nearest neighbor algorithm (only one neighbor and not many neighbors)

### Nearest neighbor
The idea is very simple. Let's say we have many points in R2 as the training data each of them has a label as shown in figure 1.1 below.

We are given a new point and we are to determine which label to put on that point. See figure 1.2

A simple idea is to search for the point "closest" to the given point and use that label. That is exactly the nearest neighbor algorithm. Why did we use "quotations" on the word "closest"? Because we need to define what closest means. It means the point with the least distance to our given point.

To do that we need to define a distance measure. We can use many measures. The first one that probably came to your mind was the euclidean distance calculated using the formula $$\sqrt{(x_2-x_1)^2+(y_2-y_1)^2}$$

This is one of the better distance measures to use in nearest neighbor. But there are other distance measures, some are presented below:

#### Euclidean distance
This is the one with the formula presented above generalized for more than one component
$$\sqrt{(x_2-x_1)^2+(y_2-y_1)^2+(z_2-z_1)^2....}$$

#### MSE (mean squared error)
If x = [x1,x2,x2] and y = [y1,y2,y3], $$AverageOverAllComponentsOf((x_2-x_1)^2)$$

#### RMSE (root mean squared error)
This is simply $$\sqrt{MSE}$$



## Ok, now that we are done with nearest neighbor, let's see K nearest neighbors
This is just as the previous algorithm except that instead of finding the closest point (only 1 point), we search for the neasrest K points (that why they are multiple neighbors). Then we see what label is present the most in these K points. To see why this could be a good idea let's see this example of a digit that was misclassified using nearest neighbor but using K nearest neighbors caused a correct classification.

Ok so K nearest neighbors might be better, but how do we choose K? Unfortunately, there is no good answer to this. The best way is to try our multiple K values and graph them with the error each one has. Then find the minimum of the graph to find the optimal K. However, make sure that the dataset you use to test each K value is different from the training dataset. Also make sure that after you choose K, you test the model using a test dataset the model hasn't seen previously.

In [ ]:
# Let's implement the KNN algorithm
# Note that we will prefer using classes to implement algorithm because they can be reused with a wider variety of dataset

class KNN():
    def __init__(self,K,training_data,training_labels,type_of_distance_measure):
        self.K = K
        self.data_X = training_data
        self.data_y = training_labels
        
        if type_of_distance_measure=="MSE":
            self.distance_measure = self.compareMSE
        elif type_of_distance_measure=="RMSE":
            self.distance_measure = self.compareRMSE
        elif type_of_distance_measure=="Euclidean":
            self.distance_measure = self.compareEuclidean
        else:
            print("Error, {} is not defined as a distance measure!".format(type_of_distance_measure))
        
    def predict(self,x_input):
        
        # Calculating the distance between the given input point and all points in the training dataset
        distances = []
        for image in self.data_X:
            distance = self.distance_measure(x_input,image)
            distances.append(distance)
            
        # Find neasrest K points
        nearest_K_indices = np.argsort(distances)[:K]
        
        # Find the labels of those nearest K points and add them to an array
        neighbor_labels = []
        for i in nearest_K_indices:
            neighbor_label = self.data_y[sorted_indices[i]]
            neighbor_labels.append(neighbor_label)
        
        # See which labels is most presented in our K points
        values,counts = np.unique(neighbor_labels,return_counts=True)
        best_index=np.argmax(counts)
        return values[best_index]
        
    def compareMSE(self,x1,x2):
        x2 = np.array(x2)/255
        x1 = np.array(x1)/255
        diff = x1-x2
        squared_diff = diff*diff
        MSE = squared_diff.mean()
        return MSE
    
    def compareEuclidean(self,x1,x2):
        x2 = np.array(x2)/255
        x1 = np.array(x1)/255
        diff = x1-x2
        squared_diff = diff*diff
        SSE = squared_diff.sum()
        return np.sqrt(SSE)
    
    def compareRMSE(self,x1,x2):
        x2 = np.array(x2)/255
        x1 = np.array(x1)/255
        diff = x1-x2
        abs_diff = abs(diff)
        RMSE = np.mean(abs_diff)
        return RMSE

In [5]:
'''
This function is used to test our model
Inputs:
model: our model which needs to have a method "predict" that takes input and returns label
test_X: testing images
test_y: testing labels
verbose: If true, detailed accuracy, precision, recall, and f1 scores will be printed, otherwise nothing will be printed

Returns:
overall_accuracy: accuracy of the model over all classes
overall_precision: precision of the model over all classes
overall_recall: recall of the model over all classes
overall_f1: f1 score of the model over all classes
'''
def test_model(model,test_X,test_y,verbose=True):
    true_positives = np.zeros(10)
    false_positives = np.zeros(10)
    true_negatives = np.zeros(10)
    false_negatives = np.zeros(10)
    for x,y in tqdm(zip(test_X,test_y)):
        prediction = model.predict(x)
        if prediction == y:
            true_positives[y] += 1
            for i in range(10):
                if i != y:
                    true_negatives[i] += 1
        else:
            false_positives[prediction] += 1
            false_negatives[y] += 1
            for i in range(10):
                if i != y and i!= prediction:
                    true_negatives[i] += 1
    overall_accuracy = (true_positives.sum()+true_negatives.sum())/(true_positives.sum()+true_negatives.sum()+false_positives.sum()+false_negatives.sum())
    accuracies = (true_positives+true_negatives)/(true_positives+true_negatives+false_positives+false_negatives)
    
    overall_precision = true_positives.sum()/(true_positives.sum()+false_positives.sum())
    precisions = true_positives/(true_positives+false_positives)
    
    overall_recall = true_positives.sum()/(true_positives.sum()+false_negatives.sum())
    recalls = true_positives/(true_positives+false_negatives)
    
    overall_f1 = 2*(overall_recall*overall_precision)/(overall_recall+overall_precision)
    f1s = 2*(recalls*precisions)/(recalls+precisions)
    
    if verbose:
        print("Accuracy Measures:")
        for i in range(10):
            print(str(i)+" : {}%".format(accuracies[i]*100))
        print("Overall Accuracy : {}%".format(overall_accuracy*100))

        print("Precision Measures:")
        for i in range(10):
            print(str(i)+" : {}%".format(precisions[i]*100))
        print("Overall Precision : {}%".format(overall_precision*100))

        print("Recall Measures:")
        for i in range(10):
            print(str(i)+" : {}%".format(recalls[i]*100))
        print("Overall Recall : {}%".format(overall_recall*100))

        print("F1 Measures:")
        for i in range(10):
            print(str(i)+" : {}%".format(f1s[i]*100))
        print("Overall F1 : {}%".format(overall_f1*100))
    
    return overall_accuracy,overall_precision,overall_recall,overall_f1

In [ ]:
accuracies = []
precisions = []
recalls = []
f1s = []
number_of_tests = 100
Ks_to_test = [1,2,3,5,7,9,11,15,20,25,35,50]
for k in Ks_to_test:
    model = KNN(k,training_X,training_y,"Euclidean")
    accuracy,precision,recall,f1 = test_model(model,testing_X[:number_of_tests],testing_y[:number_of_tests],verbose=False)
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
    print("K={} has an accuracy={}".format(k,accuracy))

100it [03:35,  2.49s/it]


K=1 has an accuracy=1.0


100it [03:40,  2.09s/it]


K=2 has an accuracy=0.994


100it [03:48,  2.32s/it]


K=3 has an accuracy=0.998


78it [03:01,  2.26s/it]